# Advanced ANNs: Hyperparameter Tuning

In [ ]:
import tensorflow as tf
import numpy as np
import tensorflow.keras as keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
import matplotlib.pyplot as plt
import pandas as pd
import copy
import metpy
import metpy.calc

In [ ]:
# we will set the seed this time to ensure fairness
keras.utils.set_random_seed(1)

In [ ]:
al_asos = pd.read_csv("../Week_6/al_asos_jul_23_v2.csv")
us_asos = pd.read_csv("/share/share/combined_asos_2023.csv")

In [ ]:
def nws_heat_index(T, RH):
    '''
    T in fahrenheit, RH in %
    https://www.wpc.ncep.noaa.gov/html/heatindex_equation.shtml
    '''
    initial_hi =  0.5 * (T + 61.0 + ((T-68.0)*1.2) + (RH*0.094))
    if initial_hi < 80:
        return initial_hi

    adjustment_factor = 0
    if T>=80 and T<=112 and RH <13:
        adjustment_factor =  ((13-RH)/4)*np.sqrt((17-np.abs(T-95.))/17)
    
    if T>=80 and T<=87 and RH > 85:
         adjustment_factor = ((RH-85)/10) * ((87-T)/5)

    return (-42.379 + 2.04901523*T + 10.14333127*RH - .22475541*T*RH - .00683783*T*T - 
           .05481717*RH*RH + .00122874*T*T*RH + .00085282*T*RH*RH - .00000199*T*T*RH*RH) + adjustment_factor

nws_heat_index_vectorized = np.vectorize(nws_heat_index)

In [ ]:
al_asos

In [ ]:
us_asos

In [ ]:
al_asos['heat_index'] = nws_heat_index_vectorized(al_asos["tmpf"], al_asos['relh'])

In [ ]:
al_asos['heat_index'].hist()

In [ ]:
us_asos['relh_raw'] = metpy.calc.relative_humidity_from_dewpoint(us_asos['tmpf'].values*metpy.units.units("degF"), 
                                                             us_asos['dwpf'].values*metpy.units.units("degF")).magnitude*100

In [ ]:
us_asos['relh_raw'] 

In [ ]:
us_asos['relh'] = us_asos['relh_raw'].clip(0,100)

In [ ]:
us_asos['heat_index'] = nws_heat_index_vectorized(us_asos["tmpf"], us_asos['relh'])

In [ ]:
us_asos['heat_index'].hist()

In [ ]:
from sklearn.model_selection import train_test_split

### Train/test/validate on intentionally small data

In [ ]:
al_asos_nona = al_asos.dropna(subset=['tmpf', 'relh', 'heat_index'])

In [ ]:
# Prepare our data first
using_data_AL, _ = train_test_split(al_asos_nona, test_size=0.98)
training_data_AL, test_validation_AL = train_test_split(using_data_AL, test_size=0.25)
testing_data_AL, validation_data_AL = train_test_split(test_validation_AL, test_size=0.5)

In [ ]:
len(training_data_AL), len(testing_data_AL), len(validation_data_AL)

In [ ]:
import optuna

In [ ]:
tf.keras.backend.clear_session()
keras.utils.set_random_seed(1)


In [ ]:
def optuna_function(trial):
    input1 = keras.layers.Input(shape=(2, ))
    input_norm = keras.layers.Normalization(axis=1)(input1)


    dense1 = keras.layers.Dense(trial.suggest_int("neurons_dense1",4, 1024), activation=None)(input_norm)
    lerelu_layer = keras.layers.LeakyReLU(alpha=0.3)(dense1) 
    output = keras.layers.Dense(1, activation=trial.suggest_categorical('activation_output', ['relu', 'linear', 'sigmoid'])
                                )(lerelu_layer)
    model_AL = keras.models.Model(inputs=[input1], outputs=[output])
    model_AL.compile(optimizer=keras.optimizers.legacy.Adam(), loss='mean_squared_error')

    lr = trial.suggest_float('lr', 1e-6, 1e0, log=True)
    model = keras.models.Model(inputs=[input1], outputs=[output])

    model.compile(loss='mean_squared_error', optimizer=keras.optimizers.legacy.Adam(learning_rate=lr), metrics=['accuracy'])
    model.fit(training_data_AL[['tmpf', 'relh']].values, 
           training_data_AL['heat_index'].values, epochs=30, batch_size=64, verbose=False)
    return model.evaluate(validation_data_AL[['tmpf', 'relh']].values, validation_data_AL['heat_index'].values, verbose=False)[0]


In [ ]:
study = optuna.create_study()
study.optimize(optuna_function, n_trials=10)

study.best_params  # E.g. {'x': 2.002108042}
